In [1]:
import seaborn as sns

tips = sns.load_dataset('tips')

In [2]:
import pandas as pd
import numpy as np

In [3]:
tips["Car_Class"]=["A" if x<10 else "B" for x in tips["total_bill"]]
tips_OH=pd.get_dummies(tips)

In [4]:
#Finds all the positions where char is in word
def findall(char,word):
    pos=[word.find(char)]
    suma=pos[0]+1
    while pos[-1]!=-1:
        pos.append(word[suma:].find(char))
        suma+=pos[-1]+1#(hay que sumar 1 por el "_" que hay que borrar)
    if pos[0]==-1:
        return pos
    pos=pos[:-1]
    suma=pos[0]
    for i in range(1,len(pos)):
        suma+=pos[i]+1#Hay que sumar 1 por los arrays empiezan en 0)
        pos[i]=suma
    return pos

In [5]:
def De_OH_column_names(columns_df):
    """Tries to find all the columns of the original dataframe, os numerical+De_OH_encoded"""
    future_colnames=[]
    for i in range(len(columns_df)):
        pos=findall("_",columns_df[i])
        if pos==[-1] and columns_df[i] not in future_colnames:
            future_colnames.append(columns_df[i])
            continue
        
        names=[columns_df[i][:p] for p in pos]
        #We´re gonna go over all the possibilities, so each round we´ve got a longer name
        #We are gonna assume that the longest possible name is the correct name
        name_final=""
        for name in names:
            Has_a_match=False#possible_columns=[]
            long=len(name)
            for j in range(len(columns_df)):
                #We can´t check ourselves because we would just return our own OH encoded name
                if j==i:       
                    continue 
                if name==columns_df[j][:long]:
                    #If the column isn´t actually followed by a _ we don´t want it (we´re looking for other OH encoded like us, which if it exists it´ll be followed by a _)       
                    if len(columns_df[j])>long:       
                        if columns_df[j][long]!="_":       
                            continue  
                    Has_a_match=True#possible_columns.append(columns_df[j])
            if Has_a_match:#len(possible_columns)>=1:
                name_final=name
            #It´s possible that we´re alone as it´s a binary and drop first-> we take the longest name
        if name_final not in future_colnames and name_final!="":
            future_colnames.append(name_final)
        #It´s possible that we don´t find a match, and most likely it´s because we´re dealing with a numerical (not OH)
        if name_final=="" and columns_df[i] not in future_colnames:
            future_colnames.append(columns_df[i])

    return future_colnames

In [43]:
def DeOneHotEncodeColumn(df_true,colname,inplace=False):
    """This function decodes onehotencoded columns that start with colname (and eliminates those columns)"""
    if inplace:
        df=df_true.copy()
    else:
        df=df_true
    #These are the columns we think are onehotencoded
    cols_tryout=[col for col in df.columns if colname+"_"==col[:len(colname)+1]]
    #The cols from cols_tryout aren´t necessarily binary, so we´ll check
    cols=[]
    for col in cols_tryout:
        EsBin = df[col].isin([0, 1]).all()

        if EsBin:
            cols.append(col)
    
    #If there aren´t any binary columns... what are u doing lol
    if len(cols)==0:
        return df
    elif len(cols)==1:
        col=cols[0]
        sol=np.array([col[len(colname)+1:] if x else "not "+col[len(colname)+1:] for x in df[col]],dtype="object")
    else:
        sol=np.array(["" for i in range(df.shape[0])],dtype="object")
        #Let´s get the output of our decoded column ()
        for col in cols:
            sol+=np.array([col[len(colname)+1:] if x else "" for x in df[col]],dtype="object")

    #Let´s drop the columns we used and put the decoded column in place :)
    df[colname]=sol
    df=df.drop(cols,axis=1)
    return df

In [14]:
#We´re gonna detect the columns, sort their names so that EVENT_TYPE is done before EVENT...and done :)
def DeOneHotEncode(df_orig,exclude=[]):
    df=df_orig.copy()
    cols=pd.Series(De_OH_column_names(df.columns))
    cols=cols.sort_values(ascending=False)
    for col in cols:
        if col not in exclude and col not in df.columns:
            df=DeOneHotEncodeColumn(df,col,inplace=True)
    return df

In [23]:
diamonds2_OH

,carat,depth,table,price,x,y,z,cut_Ideal,cut_Premium,cut_Very Good,...,cat_column_9_A,cat_column_9_B,cat_column_9_C,cat_column_9_D,cat_column_9_E,cat_column_10_A,cat_column_10_B,cat_column_10_C,cat_column_10_D,cat_column_10_E
0,0.23,61.5,55.0,326,3.95,3.98,2.43,True,False,False,...,False,True,False,False,False,True,False,False,False,False
1,0.21,59.8,61.0,326,3.89,3.84,2.31,False,True,False,...,True,False,False,False,False,False,False,False,True,False
2,0.23,56.9,65.0,327,4.05,4.07,2.31,False,False,False,...,False,True,False,False,False,True,False,False,False,False
3,0.29,62.4,58.0,334,4.20,4.23,2.63,False,True,False,...,False,True,False,False,False,True,False,False,False,False
4,0.31,63.3,58.0,335,4.34,4.35,2.75,False,False,False,...,True,False,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53935,0.72,60.8,57.0,2757,5.75,5.76,3.50,True,False,False,...,True,False,False,False,False,False,False,False,True,False
53936,0.72,63.1,55.0,2757,5.69,5.75,3.61,False,False,False,...,False,True,False,False,False,False,False,False,False,True
53937,0.70,62.8,60.0,2757,5.66,5.68,3.56,False,False,True,...,False,False,False,False,True,True,False,False,False,False
53938,0.86,61.0,58.0,2757,6.15,6.12,3.74,False,True,False,...,False,False,True,False,False,False,False,True,False,False


In [147]:
DeOneHotEncodeColumn(tips_OH,"Car_Class")

,total_bill,tip,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner,Car_Class
0,16.99,1.01,2,False,True,False,True,False,False,False,True,False,True,B
1,10.34,1.66,3,True,False,False,True,False,False,False,True,False,True,B
2,21.01,3.50,3,True,False,False,True,False,False,False,True,False,True,B
3,23.68,3.31,2,True,False,False,True,False,False,False,True,False,True,B
4,24.59,3.61,4,False,True,False,True,False,False,False,True,False,True,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,29.03,5.92,3,True,False,False,True,False,False,True,False,False,True,B
240,27.18,2.00,2,False,True,True,False,False,False,True,False,False,True,B
241,22.67,2.00,2,True,False,True,False,False,False,True,False,False,True,B
242,17.82,1.75,2,True,False,False,True,False,False,True,False,False,True,B


In [18]:
diamonds2=diamonds.copy()
# Add 10 additional categorical columns
for i in range(1, 11):
    # Generate random categorical data for simplicity
    categories = ['A', 'B', 'C', 'D', 'E']
    diamonds2[f'cat_column_{i}'] = np.random.choice(categories, size=diamonds2.shape[0])

In [19]:
diamonds2_OH=pd.get_dummies(diamonds2)

In [17]:
import seaborn as sns

# Load the 'flights' dataset
diamonds = sns.load_dataset('diamonds')


In [28]:
%%timeit
DeOneHotEncodeColumn(diamonds2_OH,"cat_column_8")


2.05 s ± 68.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
%%timeit
DeOneHotEncodeColumn2(diamonds2_OH,"cat_column_10")


36.5 ms ± 744 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
diamonds2_OH["cat_column_10_B"][50000]=3

C:\Users\Rafa\AppData\Local\Temp\ipykernel_3620\1842597354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  diamonds2_OH["cat_column_10_B"][50000]=3


In [25]:
diamonds2.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z', 'cat_column_1', 'cat_column_2', 'cat_column_3', 'cat_column_4',
       'cat_column_5', 'cat_column_6', 'cat_column_7', 'cat_column_8',
       'cat_column_9', 'cat_column_10'],
      dtype='object')